# ETL House Sales Index - Join All and Create Ratios

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
zillow_melt_sales = pd.read_csv('zillow_sales_value_index_zip_month.csv').drop('Unnamed: 0', axis=1)
display(zillow_melt_sales.head())
complaints_counts = pd.read_csv('grouped_311_noise v02.csv').drop('Unnamed: 0', axis=1)
display(complaints_counts.head())
zip_pop = pd.read_csv('population_zip_year.csv').drop('Unnamed: 0', axis=1)
display(zip_pop.head())
irs_income = pd.read_csv('irs_income_zip.csv').drop('Unnamed: 0', axis=1)
display(irs_income.head())

,zillow_zip,zillow_month_day,zillow_sales_value_index,zillow_month
0,10001,2010-01-31,613892.359651,2010-01
1,10001,2010-02-28,607612.163912,2010-02
2,10001,2010-03-31,604503.832397,2010-03
3,10001,2010-04-30,605535.608025,2010-04
4,10001,2010-05-31,608584.460474,2010-05


,incident_zip,month,year,borough,qty_complaints,0-6 hours,7-12 hours,13-18 hours,19-24 hours,21 Collection Truck Noise,...,Noise: Loud Music/Daytime (Mark Date And Time) (NN1),Noise: Loud Music/Nighttime(Mark Date And Time) (NP1),Noise: Manufacturing Noise (NK1),Noise: Other Noise Sources (Use Comments) (NZZ),Noise: Private Carting Noise (NQ1),Noise: Vehicle (NR2),Noise: air condition/ventilation equipment (NV1),Other,Passing By,People Created Noise
0,0,2018-04,2018,BROOKLYN,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,83,2013-08,2013,MANHATTAN,25,0,0,25,0,0,...,0,0,0,0,0,0,0,0,0,0
2,83,2013-09,2013,MANHATTAN,28,0,2,26,0,0,...,0,0,0,0,0,0,0,0,0,0
3,83,2013-10,2013,MANHATTAN,21,0,2,19,0,0,...,0,0,0,0,0,0,0,0,0,0
4,83,2013-11,2013,MANHATTAN,2,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


,pop_zip_code,population,pop_year
0,601,18533.0,2011
1,602,41930.0,2011
2,603,54475.0,2011
3,606,6386.0,2011
4,610,29111.0,2011


,irs_zip_code,Total_income_A02650,Total_returns_N02650
0,0,1.397357e+08,2161240.0
1,35004,3.311830e+05,5420.0
2,35005,1.392660e+05,3440.0
3,35006,6.675500e+04,1230.0
4,35007,7.767800e+05,12600.0


In [3]:
#Extract year from zillow date
zillow_melt_sales['zillow_year'] = pd.DatetimeIndex(zillow_melt_sales['zillow_month_day']).year

In [4]:
#Fill williamsbourg missing population
for i in range(2010,2021):
    new_row = zip_pop[zip_pop['pop_zip_code']==11249].head(1)
    new_row['pop_year'] = i
    zip_pop = pd.concat([zip_pop,new_row], ignore_index=True)
zip_pop[zip_pop['pop_zip_code']==11249]

,pop_zip_code,population,pop_year
334107,11249,42421.0,2021
401025,11249,42421.0,2022
434799,11249,42421.0,2023
465690,11249,42421.0,2010
465691,11249,42421.0,2011
465692,11249,42421.0,2012
465693,11249,42421.0,2013
465694,11249,42421.0,2014
465695,11249,42421.0,2015
465696,11249,42421.0,2016


In [5]:
join_sales = pd.DataFrame()
join_sales = zillow_melt_sales.merge(complaints_counts,how='left', left_on=['zillow_zip','zillow_month'],right_on=['incident_zip','month'])
join_sales = join_sales.merge(zip_pop, how = 'left', left_on= ['zillow_zip','zillow_year'], right_on = ['pop_zip_code','pop_year'])
join_sales = join_sales.merge(irs_income, how = 'left', left_on= 'zillow_zip', right_on = 'irs_zip_code')
join_sales['Total_income_per_capita'] = join_sales['Total_income_A02650']/join_sales['population']
print(join_sales.shape)
join_sales.head()

(27946, 58)


,zillow_zip,zillow_month_day,zillow_sales_value_index,zillow_month,zillow_year,incident_zip,month,year,borough,qty_complaints,...,Other,Passing By,People Created Noise,pop_zip_code,population,pop_year,irs_zip_code,Total_income_A02650,Total_returns_N02650,Total_income_per_capita
0,10001,2010-01-31,613892.359651,2010-01,2010,10001.0,2010-01,2010.0,MANHATTAN,98.0,...,0.0,0.0,0.0,10001,21097.0,2010,10001.0,2830868.0,15590.0,134.183438
1,10001,2010-02-28,607612.163912,2010-02,2010,10001.0,2010-02,2010.0,MANHATTAN,93.0,...,0.0,0.0,0.0,10001,21097.0,2010,10001.0,2830868.0,15590.0,134.183438
2,10001,2010-03-31,604503.832397,2010-03,2010,10001.0,2010-03,2010.0,MANHATTAN,105.0,...,0.0,0.0,0.0,10001,21097.0,2010,10001.0,2830868.0,15590.0,134.183438
3,10001,2010-04-30,605535.608025,2010-04,2010,10001.0,2010-04,2010.0,MANHATTAN,143.0,...,0.0,0.0,0.0,10001,21097.0,2010,10001.0,2830868.0,15590.0,134.183438
4,10001,2010-05-31,608584.460474,2010-05,2010,10001.0,2010-05,2010.0,MANHATTAN,127.0,...,0.0,0.0,0.0,10001,21097.0,2010,10001.0,2830868.0,15590.0,134.183438


In [6]:
join_sales.dropna(subset=['zillow_sales_value_index'], inplace = True)
join_sales['qty_complaints'] = join_sales['qty_complaints'].fillna(value=0)
join_sales['income_group'] = pd.qcut(join_sales['Total_income_per_capita'],q=3, labels=["low","medium","high"])
join_sales['comp_ratio_1000_total'] = join_sales['qty_complaints'] / join_sales['population']*1000
join_sales['comp_ratio_1000_0-6 hours'] = join_sales['0-6 hours'] / join_sales['population']*1000
join_sales['comp_ratio_1000_7-12 hours'] = join_sales['7-12 hours'] / join_sales['population']*1000
join_sales['comp_ratio_1000_13-18 hours'] = join_sales['13-18 hours'] / join_sales['population']*1000
join_sales['comp_ratio_1000_19-24 hours'] = join_sales['19-24 hours'] / join_sales['population']*1000
join_sales['comp_ratio_1000_21 Collection Truck Noise'] = join_sales['21 Collection Truck Noise'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Banging/Pounding'] = join_sales['Banging/Pounding'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Car/Truck Horn'] = join_sales['Car/Truck Horn'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Car/Truck Music'] = join_sales['Car/Truck Music'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Engine Idling'] = join_sales['Engine Idling'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Flying Too Low'] = join_sales['Flying Too Low'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Horn Honking Sign Requested (NR9)'] = join_sales['Horn Honking Sign Requested (NR9)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Hovering'] = join_sales['Hovering'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Loud Music/Party'] = join_sales['Loud Music/Party'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Loud Talking'] = join_sales['Loud Talking'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Loud Television'] = join_sales['Loud Television'] / join_sales['population']*1000
join_sales['comp_ratio_1000_NYPD'] = join_sales['NYPD'] / join_sales['population']*1000
join_sales['comp_ratio_1000_News Gathering'] = join_sales['News Gathering'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise'] = join_sales['Noise'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise, Barking Dog (NR5)'] = join_sales['Noise, Barking Dog (NR5)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise, Ice Cream Truck (NR4)'] = join_sales['Noise, Ice Cream Truck (NR4)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise, Other Animals (NR6)'] = join_sales['Noise, Other Animals (NR6)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise:  lawn care equipment (NCL)'] = join_sales['Noise:  lawn care equipment (NCL)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Air Condition/Ventilation Equip, Commercial (NJ2)'] = join_sales['Noise: Air Condition/Ventilation Equip, Commercial (NJ2)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Air Condition/Ventilation Equip, Residential (NJ1)'] = join_sales['Noise: Air Condition/Ventilation Equip, Residential (NJ1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Alarms (NR3)'] = join_sales['Noise: Alarms (NR3)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Boat(Engine'] = join_sales['Noise: Boat(Engine'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Boat(Engine,Music,Etc) (NR10)'] = join_sales['Noise: Boat(Engine,Music,Etc) (NR10)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Construction Before/After Hours (NM1)'] = join_sales['Noise: Construction Before/After Hours (NM1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Construction Equipment (NC1)'] = join_sales['Noise: Construction Equipment (NC1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Jack Hammering (NC2)'] = join_sales['Noise: Jack Hammering (NC2)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Loud Music From Siebel System - For Dep Internal Use Only (NP21)'] = join_sales['Noise: Loud Music From Siebel System - For Dep Internal Use Only (NP21)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Loud Music/Daytime (Mark Date And Time) (NN1)'] = join_sales['Noise: Loud Music/Daytime (Mark Date And Time) (NN1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Loud Music/Nighttime(Mark Date And Time) (NP1)'] = join_sales['Noise: Loud Music/Nighttime(Mark Date And Time) (NP1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Manufacturing Noise (NK1)'] = join_sales['Noise: Manufacturing Noise (NK1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Other Noise Sources (Use Comments) (NZZ)'] = join_sales['Noise: Other Noise Sources (Use Comments) (NZZ)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Private Carting Noise (NQ1)'] = join_sales['Noise: Private Carting Noise (NQ1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: Vehicle (NR2)'] = join_sales['Noise: Vehicle (NR2)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Noise: air condition/ventilation equipment (NV1)'] = join_sales['Noise: air condition/ventilation equipment (NV1)'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Other'] = join_sales['Other'] / join_sales['population']*1000
join_sales['comp_ratio_1000_Passing By'] = join_sales['Passing By'] / join_sales['population']*1000
join_sales['comp_ratio_1000_People Created Noise'] = join_sales['People Created Noise'] / join_sales['population']*1000
print(join_sales.shape)
join_sales.head()

(27482, 101)


,zillow_zip,zillow_month_day,zillow_sales_value_index,zillow_month,zillow_year,incident_zip,month,year,borough,qty_complaints,...,comp_ratio_1000_Noise: Loud Music/Daytime (Mark Date And Time) (NN1),comp_ratio_1000_Noise: Loud Music/Nighttime(Mark Date And Time) (NP1),comp_ratio_1000_Noise: Manufacturing Noise (NK1),comp_ratio_1000_Noise: Other Noise Sources (Use Comments) (NZZ),comp_ratio_1000_Noise: Private Carting Noise (NQ1),comp_ratio_1000_Noise: Vehicle (NR2),comp_ratio_1000_Noise: air condition/ventilation equipment (NV1),comp_ratio_1000_Other,comp_ratio_1000_Passing By,comp_ratio_1000_People Created Noise
0,10001,2010-01-31,613892.359651,2010-01,2010,10001.0,2010-01,2010.0,MANHATTAN,98.0,...,0.0,0.0,0.0,0.0474,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,2010-02-28,607612.163912,2010-02,2010,10001.0,2010-02,2010.0,MANHATTAN,93.0,...,0.0,0.0,0.0,0.0474,0.0,0.0,0.0,0.0,0.0,0.0
2,10001,2010-03-31,604503.832397,2010-03,2010,10001.0,2010-03,2010.0,MANHATTAN,105.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0
3,10001,2010-04-30,605535.608025,2010-04,2010,10001.0,2010-04,2010.0,MANHATTAN,143.0,...,0.0,0.0,0.0,0.0474,0.0,0.0,0.0,0.0,0.0,0.0
4,10001,2010-05-31,608584.460474,2010-05,2010,10001.0,2010-05,2010.0,MANHATTAN,127.0,...,0.0,0.0,0.0,0.0948,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
join_sales = join_sales.join(pd.get_dummies(join_sales['zillow_zip'], prefix = 'zip'))
print(join_sales.shape)
join_sales.head()

(27482, 279)


,zillow_zip,zillow_month_day,zillow_sales_value_index,zillow_month,zillow_year,incident_zip,month,year,borough,qty_complaints,...,zip_11429,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436,zip_11691,zip_11692,zip_11693,zip_11694
0,10001,2010-01-31,613892.359651,2010-01,2010,10001.0,2010-01,2010.0,MANHATTAN,98.0,...,0,0,0,0,0,0,0,0,0,0
1,10001,2010-02-28,607612.163912,2010-02,2010,10001.0,2010-02,2010.0,MANHATTAN,93.0,...,0,0,0,0,0,0,0,0,0,0
2,10001,2010-03-31,604503.832397,2010-03,2010,10001.0,2010-03,2010.0,MANHATTAN,105.0,...,0,0,0,0,0,0,0,0,0,0
3,10001,2010-04-30,605535.608025,2010-04,2010,10001.0,2010-04,2010.0,MANHATTAN,143.0,...,0,0,0,0,0,0,0,0,0,0
4,10001,2010-05-31,608584.460474,2010-05,2010,10001.0,2010-05,2010.0,MANHATTAN,127.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
join_sales[join_sales['comp_ratio_1000_total'].isna()]

,zillow_zip,zillow_month_day,zillow_sales_value_index,zillow_month,zillow_year,incident_zip,month,year,borough,qty_complaints,...,zip_11429,zip_11432,zip_11433,zip_11434,zip_11435,zip_11436,zip_11691,zip_11692,zip_11693,zip_11694


In [9]:
join_sales.to_csv('join_sales_index_datasets.csv', index = False)